# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#List-Exogenous-SymPy-Variables"><span class="toc-item-num">3 - </span>List Exogenous <code>SymPy</code> Variables</a></div><div class="lev1"><a href="#Build-the-Structure-of-the-Valuation-Model"><span class="toc-item-num">4 - </span>Build the Structure of the Valuation Model</a></div><div class="lev1"><a href="#Calculate-Actual-Values"><span class="toc-item-num">5 - </span>Calculate Actual Values</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas
from pprint import pprint
import sympy

from HelpyFuncs.SymPy import numpy_vector, sympy_eval_by_theano
    
from HelpyFuncs.SymPy.Finance import \
    terminal_value as tv, \
    present_value as pv, \
    net_present_value as npv

Using gpu device 0: GeForce GT 750M


# Set _CONSTANTS_

In [2]:
# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .48

# Risk-Free Rate
RISK_FREE_RATE = .11

# Public Market Return
PUB_MKT_RETURN = .19

# CAPM Betas
PUB_MKT_BETA_FOR_FIN_SECTOR = 1.   # assumed in line with overall equity market
PEVC_BETA_FOR_PRO_FORMA_PERIOD = 2.

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .08

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 3

# List Exogenous `SymPy` Variables

In [3]:
# Cash Flow components
Rev = sympy.MatrixSymbol('Rev', NB_PRO_FORMA_YEARS + 1, 1)
OpEx___neg = sympy.MatrixSymbol('OpEx___neg', NB_PRO_FORMA_YEARS + 1, 1)
extra_exec_comp___neg = sympy.MatrixSymbol('extra_exec_comp___neg', NB_PRO_FORMA_YEARS + 1, 1)

# Corporate Tax Rate
corp_tax_rate = sympy.Symbol('corp_tax_rate')

# Long-Term Growth Rate
long_term_growth_rate = sympy.Symbol('long_term_growth_rate')

# CAPM Paramaters
risk_free_rate = sympy.Symbol('risk_free_rate')
pub_mkt_return = sympy.Symbol('pub_mkt_return')
stabilized_beta = sympy.Symbol('stabilized_beta')
pevc_beta = sympy.Symbol('pevc_beta')

# assemble all exogenous variables into a dict
exo_vars = \
    [Rev, OpEx___neg, extra_exec_comp___neg,
     corp_tax_rate,
     long_term_growth_rate,
     risk_free_rate, pub_mkt_return,
     stabilized_beta, pevc_beta]

# Build the Structure of the Valuation Model

EBIT from Revenue, Operating Expenses and extra executive compensation:

In [4]:
EBIT = Rev + OpEx___neg + extra_exec_comp___neg

print('EBIT =', EBIT)

EBIT = OpEx___neg + Rev + extra_exec_comp___neg


EBIAT after corporate tax:

In [5]:
EBIAT = (1 - corp_tax_rate) * EBIT

print('EBIAT =', EBIAT)

EBIAT = (-corp_tax_rate + 1)*(OpEx___neg + Rev + extra_exec_comp___neg)


As the case provides no details on projected investments / divestments and changes in working capital, we assume that the FCF before Terminal Value equals the estimated EBIAT:

In [6]:
FCF_before_TV = EBIAT

print('FCF_before_TV =', FCF_before_TV)

FCF_before_TV = (-corp_tax_rate + 1)*(OpEx___neg + Rev + extra_exec_comp___neg)


Discount Rates:

In [7]:
pro_forma_period_discount_rate = \
    risk_free_rate + pevc_beta * (pub_mkt_return - risk_free_rate)

print('Pro-Forma Period Discount Rate =', pro_forma_period_discount_rate)

Pro-Forma Period Discount Rate = pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate


In [8]:
long_term_discount_rate = \
    risk_free_rate + stabilized_beta * (pub_mkt_return - risk_free_rate)
    
print('Long-Term Discount Rate =', long_term_discount_rate)

Long-Term Discount Rate = risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate)


Terminal Value:

In [9]:
TerminalValue = tv(FCF_before_TV, long_term_discount_rate, long_term_growth_rate)
    
print('TerminalValue =', TerminalValue)

TerminalValue = (long_term_growth_rate + 1)*Determinant((-corp_tax_rate + 1)*Matrix([[0, 0, 0, 1]])*(OpEx___neg + Rev + extra_exec_comp___neg))/(-long_term_growth_rate + risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate))


Valuation from applying relevant discount rates to _pro forma_ period CFs and to Terminal Value:

In [10]:
val_of_FCF_before_TV = npv(FCF_before_TV, pro_forma_period_discount_rate)

val_of_terminal_val = pv(TerminalValue, long_term_discount_rate, NB_PRO_FORMA_YEARS)

val = val_of_FCF_before_TV + val_of_terminal_val

print('Valuation =', val)

Valuation = (long_term_growth_rate + 1)*Determinant((-corp_tax_rate + 1)*Matrix([[0, 0, 0, 1]])*(OpEx___neg + Rev + extra_exec_comp___neg))/((-long_term_growth_rate + risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate))*(risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate) + 1)**3) + Determinant((-corp_tax_rate + 1)*Matrix([[1, 1/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + 1), (pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + 1)**(-2), (pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + 1)**(-3)]])*(OpEx___neg + Rev + extra_exec_comp___neg))


# Calculate Actual Values

In [11]:
exo_vars_values_dict = \
    dict(
        Rev=numpy_vector([0, 288000, 799350, 1181700]),
        OpEx___neg=numpy_vector([0, -242774, -308991, -410010]),
        extra_exec_comp___neg=numpy_vector([0, -2500, -75000, -140000]),
        corp_tax_rate=US_CORP_TAX_RATE,
        risk_free_rate=RISK_FREE_RATE,
        pub_mkt_return=PUB_MKT_RETURN,
        pevc_beta=PEVC_BETA_FOR_PRO_FORMA_PERIOD,
        stabilized_beta=PUB_MKT_BETA_FOR_FIN_SECTOR,
        long_term_growth_rate=LONG_TERM_GROWTH_RATE)
    
pprint(exo_vars_values_dict)

{'OpEx___neg': array([[      0.],
       [-242774.],
       [-308991.],
       [-410010.]], dtype=float32),
 'Rev': array([[       0.],
       [  288000.],
       [  799350.],
       [ 1181700.]], dtype=float32),
 'corp_tax_rate': 0.48,
 'extra_exec_comp___neg': array([[      0.],
       [  -2500.],
       [ -75000.],
       [-140000.]], dtype=float32),
 'long_term_growth_rate': 0.08,
 'pevc_beta': 2.0,
 'pub_mkt_return': 0.19,
 'risk_free_rate': 0.11,
 'stabilized_beta': 1.0}


In [12]:
# function to calculate values from SymPy expression
calc = lambda sympy_expr: \
    sympy_eval_by_theano(
        sympy_expr,
        symbols=exo_vars,
        **exo_vars_values_dict)

In [13]:
# populate Valuation Calculations data frame
val_calcs_df = pandas.DataFrame(index=['Year 0', 1981, 1982, 1983])

val_calcs_df['Revenue'] = calc(Rev)
val_calcs_df['Operating Expenses'] = calc(OpEx___neg)
val_calcs_df['Adj. for Extra Exec. Comp.'] = calc(extra_exec_comp___neg)
val_calcs_df['EBIT'] = calc(EBIT)
val_calcs_df['EBIAT'] = calc(EBIAT)
val_calcs_df['FCF before Terminal Value'] = calc(FCF_before_TV)
val_calcs_df['Terminal Value'] = 0
val_calcs_df.loc[1983, 'Terminal Value'] = calc(TerminalValue)

In [14]:
# present whole calculations

print('U.S. Corporate Tax Rate =   %s\n' % '{:.0%}'.format(float(calc(corp_tax_rate))))

print('Risk-Free Rate =   %s' % '{:.0%}'.format(float(calc(risk_free_rate))))
print('Public-Market Return =   %s\n' % '{:.0%}'.format(float(calc(pub_mkt_return))))

print('Stabilized Beta =   %.0fx' % calc(stabilized_beta))
print('Long-Term Discount Rate =   %s\n' % '{:.0%}'.format(float(calc(long_term_discount_rate))))

print('PEVC Beta =   %.0fx' % calc(pevc_beta))
print('Discount Rate for Pro-Forma Period =   %s\n' % '{:.0%}'.format(float(calc(pro_forma_period_discount_rate))))

print('Valuation of FCF before TV =   $%.2f MM' % (calc(val_of_FCF_before_TV) / 1e6))
print('Valuation of TV =   $%.2f MM' % (calc(val_of_terminal_val) / 1e6))
print('\nVALUATION =   $%.2f MM\n' % (calc(val) / 1e6))

# print Valuation Calcs data frame
# ref: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html
pandas.options.display.float_format = '$ {:,.0f}'.format
print('Detailed Cash Flow Calcs:')
val_calcs_df.T

U.S. Corporate Tax Rate =   48%

Risk-Free Rate =   11%
Public-Market Return =   19%

Stabilized Beta =   1x
Long-Term Discount Rate =   19%

PEVC Beta =   2x
Discount Rate for Pro-Forma Period =   27%

Valuation of FCF before TV =   $0.31 MM
Valuation of TV =   $1.91 MM

VALUATION =   $2.23 MM

Detailed Cash Flow Calcs:


,Year 0,1981,1982,1983
Revenue,$ 0,"$ 288,000","$ 799,350","$ 1,181,700"
Operating Expenses,$ 0,"$ -242,774","$ -308,991","$ -410,010"
Adj. for Extra Exec. Comp.,$ 0,"$ -2,500","$ -75,000","$ -140,000"
EBIT,$ 0,"$ 42,726","$ 415,359","$ 631,690"
EBIAT,$ 0,"$ 22,218","$ 215,987","$ 328,479"
FCF before Terminal Value,$ 0,"$ 22,218","$ 215,987","$ 328,479"
Terminal Value,$ 0,$ 0,$ 0,"$ 3,225,064"
